Data set : https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

cleaning : https://colab.research.google.com/drive/1nwe1Ns9bJOv6zFhBXGzgNVXh9sJv52NW#scrollTo=Iw1cvXeALXDu


In [81]:
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np


In [82]:
import pandas as pd

In [83]:
from pyspark.sql import SparkSession

In [84]:
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[1]") \
    .getOrCreate()

In [85]:
movie = spark.read.csv('sample_data/movie.csv')

In [86]:
movie.count()

4809

In [87]:
new_column_names = ['movie_id','title','overview','genres','keywords','cast','crew']

In [88]:
movie = movie.toDF(*new_column_names)

In [89]:
movie.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|            overview|              genres|            keywords|                cast|                crew|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   19995|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|
|     285|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|
|  206647|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|
|   49026|The Dark Knight R...|Following the dea...|['Action', 'Crime...|['dc comics', 'cr...|['Christian Bale'...|['Christopher Nol...|
|   49529|         John Carter|John Carte

In [90]:
# movie.write.csv('movie_scikit.csv', header=True, mode='overwrite')

In [91]:
from pyspark.sql.functions import col

# Drop the "movie_id" column from the DataFrame
movie = movie.drop("movie_id")

# Show the updated DataFrame
movie.show(10)


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|              genres|            keywords|                cast|                crew|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|
|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|
|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|
|The Dark Knight R...|Following the dea...|['Action', 'Crime...|['dc comics', 'cr...|['Christian Bale'...|['Christopher Nol...|
|         John Carter|John Carter is a ...|['Action', 'Adven...|['based on novel'...|['Taylor Kitsch',..

In [92]:
from pyspark.sql.functions import monotonically_increasing_id

# Create a new DataFrame with a new "movie_id" column
movie = movie.withColumn("movie_id", monotonically_increasing_id() + 1)

# Show the final DataFrame
movie.show()


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|               title|            overview|              genres|            keywords|                cast|                crew|movie_id|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|       1|
|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|       2|
|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|       3|
|The Dark Knight R...|Following the dea...|['Action', 'Crime...|['dc comics', 'cr...|['Christian Bale'...|['Christopher Nol...|       4|
|         John Carter|John Carter is a ..

In [93]:
from pyspark.sql.functions import col

# Move the "movie_id" column to the leftmost position
movie = movie.select("movie_id", *[col for col in movie.columns if col != "movie_id"])

# Show the updated DataFrame
movie.show(10)


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|            overview|              genres|            keywords|                cast|                crew|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       1|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|
|       2|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|
|       3|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|
|       4|The Dark Knight R...|Following the dea...|['Action', 'Crime...|['dc comics', 'cr...|['Christian Bale'...|['Christopher Nol...|
|       5|         John Carter|John Carte

In [94]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.functions import udf, col, when
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import countDistinct
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer, RegexTokenizer, CountVectorizer, IDF
from pyspark.mllib.feature import IDF

from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [95]:
concat_udf = F.udf(lambda cols: " ".join([x if x is not None else "*" for x in cols]), StringType())


In [96]:
movie_datacont = movie.withColumn("desc", concat_udf(F.array('genres','keywords','cast')))
movie = movie_datacont.withColumn("desc", F.regexp_replace("desc", "[/(,)]", " "))

In [97]:
from pyspark.sql.functions import regexp_replace


In [98]:
movie.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|            overview|              genres|            keywords|                cast|                crew|                desc|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       1|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|['Action'  'Adven...|
|       2|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|['Adventure'  'Fa...|
|       3|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|['Action'  'Adven...|
|       4|The Dark Knight R...|Following the dea...|

In [99]:
movie = movie_datacont.withColumn('desc', regexp_replace(col('desc'), '[^A-Za-z]+',' '))

In [100]:
movie.select('desc').show(2, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|desc                                                                                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe ali

In [101]:
from pyspark.sql.functions import lower

In [102]:
# movie = movie.withColumn('desc', lower(col('desc')))

In [103]:
regexTokenizer = RegexTokenizer(inputCol="desc", outputCol="tokens", pattern='\\W')
movie = regexTokenizer.transform(movie)

In [104]:
movie.select('tokens').show(2, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tokens                                                                                                                                                                                                                                                                                                                            |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[action, adventure, fant

In [105]:
movie.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|            overview|              genres|            keywords|                cast|                crew|                desc|              tokens|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       1|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']| Action Adventure...|[action, adventur...|
|       2|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'| Adventure Fantas...|[adventure, fanta...|
|       3|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Cr

In [106]:
tfv = TfidfVectorizer(min_df = 3, max_features = None, strip_accents = 'unicode', analyzer = 'word', token_pattern =r'\w{1,}',ngram_range = (1,2),stop_words = 'english')

In [107]:
docs = movie.select('tokens').rdd.map(lambda x: x[0]).collect()
docs = [str(doc).lower() for doc in docs]

tfidf_matrix = tfv.fit_transform(docs)

cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [108]:
tfidf_matrix

<4809x8640 sparse matrix of type '<class 'numpy.float64'>'
	with 109743 stored elements in Compressed Sparse Row format>

In [109]:
# cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [110]:
# cos_sim[index]

In [111]:
# cos_sim

In [112]:
# print(cos_sim)

In [113]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType
import numpy as np

def recommendation_by_id(index):
    cosine_score = list(enumerate(cos_sim[index]))
    cosine_score = sorted(cosine_score, key=lambda x: x[1], reverse=True)
    recommendations = cosine_score[1:11]  # Adjusted to retrieve top 10 recommendations
    recommend = [i[0] for i in recommendations]
    cosine_scores = [float(i[1]) for i in recommendations]
    recom = spark.createDataFrame(zip(recommend, cosine_scores), schema=["movie_id", "cosine_score"])
    recommended_movies = recom.join(movie, recom.movie_id == movie.movie_id, 'inner').select(recom.movie_id, movie.title, recom.cosine_score)
    recommended_movies.show()



In [114]:
# Replace the index value with the desired movie index
index = 12
movie_info = movie.filter(movie['movie_id'] == index).select("movie_id", "title").first()
print("Movie ID:", movie_info.movie_id)
print("Movie Title:", movie_info.title)
print("\n")
print("Recommendation\n")
recommendation_by_id(index)


Movie ID: 12
Movie Title: Quantum of Solace


Recommendation

+--------+-------------------+-------------------+
|movie_id|              title|       cosine_score|
+--------+-------------------+-------------------+
|     199|           R.I.P.D.| 0.5511164694978855|
|       1|             Avatar| 0.4478383023015274|
|     536|       First Knight| 0.2380387538123488|
|      17|       The Avengers|0.20564322485295897|
|    2508|Hachi: A Dog's Tale|0.18779167895057047|
|     340|    The Incredibles|0.18538845541857807|
|     208|       Total Recall|0.17617671067998952|
|    3287|The Secret of Kells|0.16337214945689416|
|    1664| An Unfinished Life| 0.1540285659016967|
|    4219|Dil Jo Bhi Kahey...| 0.1406935075131897|
+--------+-------------------+-------------------+



In [115]:
# cosine_score = list(enumerate(cos_sim[index]))
# cosine_score = sorted(cosine_score, key=lambda x: x[1], reverse=True)

In [116]:
# len(cosine_score)

In [117]:
# cosine_score

In [118]:
def recommendation_by_title(title):
    movie_info = movie.filter(movie['title'] == title).select("movie_id", "title").first()
    if movie_info is None:
        print("Movie not found.")
        return

    index = int(movie_info.movie_id)  # Convert to integer

    cosine_score = list(enumerate(cos_sim[index]))
    cosine_score = sorted(cosine_score, key=lambda x: x[1], reverse=True)
    # Top 10 recommendations based on content
    recommendations = cosine_score[1:11]  # Adjusted to retrieve top 10 recommendations
    recommend = [i[0] for i in recommendations]
    cosine_scores = [float(i[1]) for i in recommendations]
    recom = spark.createDataFrame(zip(recommend, cosine_scores), schema=["movie_id", "cosine_score"])
    recommended_movies = recom.join(movie, recom.movie_id == movie.movie_id, 'inner').select(recom.movie_id, movie.title, recom.cosine_score)
    recommended_movies.show()




In [119]:
title = 'Mortdecai'

print("\n")
print("Recommendation\n")
recommendation_by_title(title)



Recommendation

+--------+--------------------+-------------------+
|movie_id|               title|       cosine_score|
+--------+--------------------+-------------------+
|     115|Harry Potter and ...| 0.4098702360384555|
|    3175|  The Purge: Anarchy| 0.3842436754335741|
|    3539|         Ghost World|0.37919125952129973|
|    1218|         Ri¢hie Ri¢h|0.37274962209154844|
|     869|       Elizabethtown| 0.3555224957038797|
|      30|             Skyfall| 0.3508976062290127|
|    2035|          Son of God|0.34327128324370637|
|    2880|          In the Cut|0.34090631100633023|
|    3380|The Other Side of...| 0.3384665383144358|
|    1034|         Mary Reilly| 0.3250415164447016|
+--------+--------------------+-------------------+

